# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv
import joblib
import numpy as np
import pandas as pd
import pkg_resources
import azureml.core
import pkg_resources
import xgboost


from matplotlib import pyplot as plt
from sklearn import datasets
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
from azureml.core.environment import Environment 
from azureml.core.model import InferenceConfig 
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.57.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-268555
aml-quickstarts-268555
westeurope
aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee


In [3]:
experiment_name = 'CapstoneExp'
project_folder = './Capstone-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
CapstoneExp,quick-starts-ws-268555,Link to Azure Machine Learning studio,Link to Documentation


In [4]:
#from azureml.data.dataset_factory import TabularDatasetFactory
#example_data = 'https://github.com/duclinh/Capstone-Azure-Machine-Learning/blob/main/student-dropout-and-academeic-success-dataset.csv'
#dataset = TabularDatasetFactory.from_delimited_files(example_data)  

found = False
key = "capstone-dataset"
description_text = "The UCI Machine Learning Repository - Predict Students' Dropout and Academic success"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
    
if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://github.com/duclinh/Capstone-Azure-Machine-Learning/blob/main/student-dropout-and-academeic-success-dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

df = dataset.to_pandas_dataframe()
df.describe()   


{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,Marital_status,Application_mode,Application_order,Course,Daytime_attendance,Previous_qualification,Previous_qualification_grade,Nacionality,Mother_qualification,Father_qualification,...,Curricularunits1stsem_withoutevaluations,Curricularunits2ndsem_credited,Curricularunits2ndsem_enrolled,Curricularunits2ndsem_evaluations,Curricularunits2ndsem_approved,Curricularunits2ndsem_grade,Curricularunits2ndsem_withoutevaluations,Unemploymentrate,Inflationrate,GDP
count,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,...,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000
mean,1.178571,18.669078,1.727848,8856.642631,0.890823,4.577758,132.613314,1.873192,19.561935,22.275316,...,0.137658,0.541817,6.232143,8.063291,4.435805,10.230206,0.150316,11.566139,1.228029,0.001969
std,0.605747,17.484682,1.313793,2063.566416,0.311897,10.216592,13.188332,6.914514,15.603186,15.343108,...,0.690880,1.918546,2.195951,3.947951,3.014764,5.210808,0.753774,2.663850,1.382711,2.269935
min,1.000000,1.000000,0.000000,33.000000,0.000000,1.000000,95.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.600000,-0.800000,-4.060000
25%,1.000000,1.000000,1.000000,9085.000000,1.000000,1.000000,125.000000,1.000000,2.000000,3.000000,...,0.000000,0.000000,5.000000,6.000000,2.000000,10.750000,0.000000,9.400000,0.300000,-1.700000
50%,1.000000,17.000000,1.000000,9238.000000,1.000000,1.000000,133.100000,1.000000,19.000000,19.000000,...,0.000000,0.000000,6.000000,8.000000,5.000000,12.200000,0.000000,11.100000,1.400000,0.320000
75%,1.000000,39.000000,2.000000,9556.000000,1.000000,1.000000,140.000000,1.000000,37.000000,37.000000,...,0.000000,0.000000,7.000000,10.000000,6.000000,13.333333,0.000000,13.900000,2.600000,1.790000
max,6.000000,57.000000,9.000000,9991.000000,1.000000,43.000000,190.000000,109.000000,44.000000,44.000000,...,12.000000,19.000000,23.000000,33.000000,20.000000,18.571429,12.000000,16.200000,3.700000,3.510000


In [5]:
dataset.take(5).to_pandas_dataframe()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,Marital_status,Application_mode,Application_order,Course,Daytime_attendance,Previous_qualification,Previous_qualification_grade,Nacionality,Mother_qualification,Father_qualification,...,Curricularunits2ndsem_credited,Curricularunits2ndsem_enrolled,Curricularunits2ndsem_evaluations,Curricularunits2ndsem_approved,Curricularunits2ndsem_grade,Curricularunits2ndsem_withoutevaluations,Unemploymentrate,Inflationrate,GDP,Target
0,1,17,5,171,1,1,122.0,1,19,12,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,17,2,9773,1,1,122.0,1,38,37,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,39,1,8014,0,1,100.0,1,37,38,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


Comuter cluster name

In [6]:
amlcompute_cluster_name = "capstone-auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2',
                                                           max_nodes=5)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded.....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
# TODO: Put your automl settings here
automl_settings = {
                    "n_cross_validations": 5,
                    "primary_metric": 'accuracy',
                    # "primary_metric" : 'AUC_weighted'
                    "enable_early_stopping": True,
                    "max_concurrent_iterations": 4,
                    "experiment_timeout_minutes": 20,
                    "verbosity": logging.INFO
}


# TODO: Put your automl config here
automl_config = AutoMLConfig(
                    compute_target = compute_target,
                    task='classification',
                    training_data=dataset,
                    label_column_name='Target',
                    path = project_folder,
                    featurization= 'auto',
                    debug_log = "automl_errors.log",
                    **automl_settings
)

In [8]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output= True)

Submitting remote run.
No run_configuration provided, running on capstone-auto-ml with default configuration
Running on remote compute: capstone-auto-ml


Experiment,Id,Type,Status,Details Page,Docs Page
CapstoneExp,AutoML_ae9cf23e-9348-40fd-9c09-a4fb1cd5ec0b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFea

In [9]:
remote_run.wait_for_completion()

{'runId': 'AutoML_ae9cf23e-9348-40fd-9c09-a4fb1cd5ec0b',
 'target': 'capstone-auto-ml',
 'status': 'Completed',
 'startTimeUtc': '2024-10-11T17:46:07.841752Z',
 'endTimeUtc': '2024-10-11T18:13:40.712494Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'capstone-auto-ml',
  'AMLSettingsJsonString': '{"path":null,"name":"CapstoneExp","subscription_id":"aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee","resource_group":"aml-quickstarts-268555","workspace_name":"quick-starts-ws-268555","region":"westeurope","compute_target":"capstone-aut

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(remote_run).show()
for children_run in remote_run.get_children():
    print('------')
    print(children_run)

2024-10-11 18:20:00.089374: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-11 18:20:00.953392: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-11 18:20:01.211170: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-11 18:20:03.002819: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-11 18:20:07.553741: W tensorflow/comp

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

------
Run(Experiment: CapstoneExp,
Id: AutoML_ae9cf23e-9348-40fd-9c09-a4fb1cd5ec0b_37,
Type: azureml.scriptrun,
Status: Completed)
------
Run(Experiment: CapstoneExp,
Id: AutoML_ae9cf23e-9348-40fd-9c09-a4fb1cd5ec0b_36,
Type: azureml.scriptrun,
Status: Completed)
------
Run(Experiment: CapstoneExp,
Id: AutoML_ae9cf23e-9348-40fd-9c09-a4fb1cd5ec0b_35,
Type: azureml.scriptrun,
Status: Canceled)
------
Run(Experiment: CapstoneExp,
Id: AutoML_ae9cf23e-9348-40fd-9c09-a4fb1cd5ec0b_34,
Type: azureml.scriptrun,
Status: Canceled)
------
Run(Experiment: CapstoneExp,
Id: AutoML_ae9cf23e-9348-40fd-9c09-a4fb1cd5ec0b_33,
Type: azureml.scriptrun,
Status: Completed)
------
Run(Experiment: CapstoneExp,
Id: AutoML_ae9cf23e-9348-40fd-9c09-a4fb1cd5ec0b_32,
Type: azureml.scriptrun,
Status: Canceled)
------
Run(Experiment: CapstoneExp,
Id: AutoML_ae9cf23e-9348-40fd-9c09-a4fb1cd5ec0b_30,
Type: azureml.scriptrun,
Status: Completed)
------
Run(Experiment: CapstoneExp,
Id: AutoML_ae9cf23e-9348-40fd-9c09-a4fb1cd5

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
# Get best run and model
best_run, fitted_model = remote_run.get_output()

# Print the best run
print(best_run)

Run(Experiment: CapstoneExp,
Id: AutoML_ae9cf23e-9348-40fd-9c09-a4fb1cd5ec0b_36,
Type: azureml.scriptrun,
Status: Completed)


In [12]:
best_run.get_metrics()

{'f1_score_macro': 0.7106629112227772,
 'average_precision_score_macro': 0.7719232032710376,
 'precision_score_macro': 0.7419328610114297,
 'average_precision_score_weighted': 0.8321856731569717,
 'matthews_correlation': 0.6446490182681466,
 'balanced_accuracy': 0.6993808876620278,
 'f1_score_weighted': 0.772447987065882,
 'recall_score_micro': 0.7854840606385969,
 'precision_score_weighted': 0.7740834416758495,
 'log_loss': 0.6073144535290776,
 'recall_score_macro': 0.6993808876620278,
 'precision_score_micro': 0.7854840606385969,
 'weighted_accuracy': 0.8457687851441291,
 'recall_score_weighted': 0.7854840606385969,
 'AUC_micro': 0.9179550353361561,
 'f1_score_micro': 0.7854840606385969,
 'AUC_weighted': 0.9087511667726862,
 'AUC_macro': 0.8944152787499341,
 'accuracy': 0.7854840606385969,
 'norm_macro_recall': 0.5490713314930417,
 'average_precision_score_micro': 0.860686796714657,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_ae9cf23e-9348-40fd-9c09-a4fb1cd5ec0b_

In [13]:
best_run.get_details()

{'runId': 'AutoML_ae9cf23e-9348-40fd-9c09-a4fb1cd5ec0b_36',
 'target': 'capstone-auto-ml',
 'status': 'Completed',
 'startTimeUtc': '2024-10-11T18:12:21.603488Z',
 'endTimeUtc': '2024-10-11T18:13:18.129491Z',
 'services': {},
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'CapstoneExp\',\'compute_target\':\'capstone-auto-ml\',\'subscription_id\':\'aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee\',\'region\':\'westeurope\',\'spark_service\':None}","ensemble_run_id":"AutoML_ae9cf23e-9348-40fd-9c09-a4fb1cd5ec0b_36","experiment_name":"CapstoneExp","workspace_name":"quick-starts-ws-268555","subscription_id":"aa7cf8e8

In [16]:
# Get all metrics of the best run
best_run_metrics = best_run.get_metrics()

In [17]:
#TODO: Save the best model
myModel = best_run.register_model(model_path='outputs/model.pkl', model_name='capstoneModel_automl',
                        tags={'Training context':'Auto ML'},
                        properties={'Accuracy': best_run_metrics['accuracy']})

print(myModel)

Model(workspace=Workspace.create(name='quick-starts-ws-268555', subscription_id='aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee', resource_group='aml-quickstarts-268555'), name=capstoneModel_automl, id=capstoneModel_automl:1, version=1, tags={'Training context': 'Auto ML'}, properties={'Accuracy': '0.7854840606385969'})


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [14]:
model = remote_run.register_model(model_name = 'best_run_automl.pkl')

environment = best_run.get_environment()
entry_script='inference/scoring.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', entry_script)


inference_config = InferenceConfig(entry_script = entry_script, environment = environment)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                    memory_gb = 1, 
                                                    auth_enabled= True, 
                                                    enable_app_insights= True)

service = Model.deploy(ws, "aciservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-10-11 18:23:33+00:00 Creating Container Registry if not exists..
2024-10-11 18:33:35+00:00 Registering the environment..
2024-10-11 18:33:36+00:00 Generating deployment configuration.
2024-10-11 18:33:38+00:00 Submitting deployment to compute..
2024-10-11 18:33:43+00:00 Checking the status of deployment aciservice..
2024-10-11 18:38:48+00:00 Checking the status of inference endpoint aciservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [18]:
print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)

Healthy
http://0616a3af-404e-41c0-b284-6ca49a055f17.westeurope.azurecontainer.io/score
http://0616a3af-404e-41c0-b284-6ca49a055f17.westeurope.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [21]:
# %run endpoint.py
import json

#Import test data
test_df = df.sample(5) # dataset is the pandas dataframe of the original data
label_df = test_df.pop('Target')

test_sample = json.dumps({'data': test_df.to_dict(orient='records')})

print(test_sample)

{"data": [{"Marital_status": 1, "Application_mode": 1, "Application_order": 4, "Course": 9500, "Daytime_attendance": 1, "Previous_qualification": 1, "Previous_qualification_grade": 137.0, "Nacionality": 1, "Mother_qualification": 37, "Father_qualification": 37, "Mother_occupation": 9, "Father_occupation": 9, "Admissiongrade": 127.5, "Displaced": 1, "Educationalspecialneeds": 0, "Debtor": 0, "Tuitionfeesuptodate": 1, "Gender": 0, "Scholarshipholder": 0, "Ageatenrollment": 18, "International": 0, "Curricularunits1stsem_credited": 0, "Curricularunits1stsem_enrolled": 8, "Curricularunits1stsem_evaluations": 8, "Curricularunits1stsem_approved": 8, "Curricularunits1stsem_grade": 13.045, "Curricularunits1stsem_withoutevaluations": 0, "Curricularunits2ndsem_credited": 0, "Curricularunits2ndsem_enrolled": 8, "Curricularunits2ndsem_evaluations": 8, "Curricularunits2ndsem_approved": 8, "Curricularunits2ndsem_grade": 13.045, "Curricularunits2ndsem_withoutevaluations": 0, "Unemploymentrate": 9.4, "

In [26]:
import requests # Used for http post request

key = 'blCzcjFSaTZ36SuKjLX5otSPGlfo28Qa'
# Set the content type
headers = {'Content-type': 'application/json'}
headers['Authorization'] = f'Bearer {key}'
response = requests.post(service.scoring_uri, test_sample, headers=headers)

In [27]:
# Print results from the inference
print(response.text)

"{\"result\": [\"Graduate\", \"Dropout\", \"Dropout\", \"Graduate\", \"Graduate\"]}"


In [28]:
# Print original labels
print(label_df)

2771    Graduate
1864     Dropout
3286     Dropout
279     Graduate
2174    Graduate
Name: Target, dtype: object


TODO: In the cell below, print the logs of the web service and delete the service

In [29]:
print(service.get_logs())

2024-10-11T18:38:32,441133700+00:00 - rsyslog/run 
2024-10-11T18:38:32,449318700+00:00 - gunicorn/run 
2024-10-11T18:38:32,464879800+00:00 | gunicorn/run | 
2024-10-11T18:38:32,467702500+00:00 | gunicorn/run | ###############################################
2024-10-11T18:38:32,472694900+00:00 | gunicorn/run | AzureML Container Runtime Information
2024-10-11T18:38:32,474516200+00:00 | gunicorn/run | ###############################################
2024-10-11T18:38:32,480387600+00:00 | gunicorn/run | 
2024-10-11T18:38:32,484681700+00:00 | gunicorn/run | 
2024-10-11T18:38:32,493223000+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20240918.v1
2024-10-11T18:38:32,498856300+00:00 - nginx/run 
2024-10-11T18:38:32,499854000+00:00 | gunicorn/run | 
2024-10-11T18:38:32,506441500+00:00 | gunicorn/run | 
2024-10-11T18:38:32,512761000+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/usr/local/sbi

In [ ]:
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
